In [4]:
#Imported Libraries

%matplotlib qt
import yaml
yaml.warnings({'YAMLLoadWarning': False})
# the default loader is deprecated, I dont know how to change the default loader
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import random
import pandas as pd
from matplotlib import colors
import seaborn as sns
from mpl_toolkits.basemap import Basemap

In [5]:
# Reading the files 
years = np.arange(1979, 2019)
file_path = r"C:\Users\krish\Documents\Data_era5\input_raw"

# 2m temperature

temperature_datasets = []
for year in years:
    whole_file_path = file_path +r"\2t\2t_%d.nc"%(year)
    dataset = xr.open_dataset(whole_file_path, chunks = {'time':10})
    dataset = dataset.resample(time='D').mean(dim='time')
    dataset = dataset.resample(time='M').mean(dim='time')
    temperature_datasets.append(dataset)
dataset_2t = xr.concat(temperature_datasets, 'time', data_vars = 'minimal')

# 2m temperature

soil_moisture_1_datasets = []
for year in years:
    whole_file_path = file_path +r"\wl1\wl1_%d.nc"%(year)
    dataset = xr.open_dataset(whole_file_path, chunks = {'time':10})
    dataset = dataset.resample(time='D').mean(dim='time')
    dataset = dataset.resample(time='M').mean(dim='time')
    soil_moisture_1_datasets.append(dataset)
dataset_wl1 = xr.concat(soil_moisture_1_datasets, 'time', data_vars = 'minimal')

# 2m temperature

soil_moisture_2_datasets = []
for year in years:
    whole_file_path = file_path +r"\wl2\wl2_%d.nc"%(year)
    dataset = xr.open_dataset(whole_file_path, chunks = {'time':10})
    dataset = dataset.resample(time='D').mean(dim='time')
    dataset = dataset.resample(time='M').mean(dim='time')
    soil_moisture_2_datasets.append(dataset)
dataset_wl2 = xr.concat(soil_moisture_2_datasets, 'time', data_vars = 'minimal')


In [6]:
# Description of the dataset
# 2m Temperature
print(dataset_2t)

# Water layer 1
print(dataset_wl1)

# Water later 2
print(dataset_wl2)

<xarray.Dataset>
Dimensions:    (latitude: 241, longitude: 480, time: 120)
Coordinates:
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 ... 357.75 358.5 359.25
  * latitude   (latitude) float32 90.0 89.25 88.5 87.75 ... -88.5 -89.25 -90.0
  * time       (time) datetime64[ns] 1979-06-30 1979-07-31 ... 2018-08-31
Data variables:
    t2m        (time, latitude, longitude) float32 dask.array<shape=(120, 241, 480), chunksize=(1, 241, 480)>
<xarray.Dataset>
Dimensions:    (latitude: 241, longitude: 480, time: 120)
Coordinates:
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 ... 357.75 358.5 359.25
  * latitude   (latitude) float32 90.0 89.25 88.5 87.75 ... -88.5 -89.25 -90.0
  * time       (time) datetime64[ns] 1979-06-30 1979-07-31 ... 2018-08-31
Data variables:
    swvl1      (time, latitude, longitude) float32 dask.array<shape=(120, 241, 480), chunksize=(1, 241, 480)>
<xarray.Dataset>
Dimensions:    (latitude: 241, longitude: 480, time: 120)
Coordinates:
  * longitude  (longitud

In [7]:
# Processing the dataset to monthly values for northern hemisphere.

#2t
dataset_2t_map = dataset_2t.sel(latitude = slice(90,25))
print(dataset_2t_map)

#wl1
dataset_wl1_map = dataset_wl1.sel(latitude = slice(90,25))
print(dataset_wl1_map)

#wl2
dataset_wl2_map = dataset_wl2.sel(latitude = slice(90,25))
print(dataset_wl2_map)

<xarray.Dataset>
Dimensions:    (latitude: 87, longitude: 480, time: 120)
Coordinates:
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 ... 357.75 358.5 359.25
  * latitude   (latitude) float32 90.0 89.25 88.5 87.75 ... 27.0 26.25 25.5
  * time       (time) datetime64[ns] 1979-06-30 1979-07-31 ... 2018-08-31
Data variables:
    t2m        (time, latitude, longitude) float32 dask.array<shape=(120, 87, 480), chunksize=(1, 87, 480)>
<xarray.Dataset>
Dimensions:    (latitude: 87, longitude: 480, time: 120)
Coordinates:
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 ... 357.75 358.5 359.25
  * latitude   (latitude) float32 90.0 89.25 88.5 87.75 ... 27.0 26.25 25.5
  * time       (time) datetime64[ns] 1979-06-30 1979-07-31 ... 2018-08-31
Data variables:
    swvl1      (time, latitude, longitude) float32 dask.array<shape=(120, 87, 480), chunksize=(1, 87, 480)>
<xarray.Dataset>
Dimensions:    (latitude: 87, longitude: 480, time: 120)
Coordinates:
  * longitude  (longitude) float32 0.

In [8]:
# Plotting Function
def draw_on_map(datas, titles, vmin, vmax, delta, colorbar_title, split = 3, sup_title = [],res=2, color='seismic'):
    """
    datas should be of length 3, arranged in the order of dataset for June, July and August
    Useful for generating climatology, trend maps, anomaly maps and composites 
    """
    plt.figure()
    if len(sup_title)!=0:
        plt.suptitle(sup_title)
    subplots = len(datas)
    
    ticks = np.arange(vmin, vmax+delta, delta)
    colors = plt.cm.get_cmap(color, len(ticks)-1)
    #ticks = np.linspace(vmin,vmax,11)
    parallels = np.arange(25,90,10.)
    meridians = np.arange(0.,360.,30.)
    for i in np.arange(subplots):
        plt.subplot(split,int(subplots/split),i+1)
        plt.title(titles[i])
        
        m = Basemap(projection='cyl',llcrnrlat=25,urcrnrlat=90,\
                llcrnrlon=0,urcrnrlon=360,resolution='c')
        m.imshow(datas[i], origin ='upper', vmin = vmin, vmax = vmax, cmap=colors)
        m.drawcoastlines()
        t_s = ticks[0::res]
        if t_s[-1]==ticks[-1]:
            cbar = m.colorbar(ticks=t_s)
        else:
            #t_s = np.append(t_s, ticks[-1])
            cbar = m.colorbar(ticks=ticks[1:-1:res])
        cbar.ax.set_title(colorbar_title)
        #for label in cbar.ax.yaxis.get_ticklabels()[::2]:
            #print(label)
         #   label.set_visible(False)
        m.drawparallels(parallels, labels = [1,0,0,0])
        m.drawmeridians(meridians, labels = [0,0,0,1])
        m.drawmapboundary(fill_color='white')
    plt.subplots_adjust(hspace=0.4)
    plt.show()
    return

In [9]:
# Importing variables for Composite Maps

%store -r july_c3_dominated 
%store -r july_c1_dominated 
%store -r july_neither_dominated
%store -r august_c3_dominated 
%store -r august_c1_dominated
%store -r august_neither_dominated

In [ ]:
# Importing c1 days and c3 days for composites

%store -r c1_days 
%store -r c3_days
%store -r c1_days_wo_jun
%store -r c3_days_wo_jun

In [10]:
august_c3_dominated

array([1983, 1988, 1990, 1993, 1998, 1999, 2002, 2007, 2009, 2011, 2014,
       2015, 2018], dtype=int64)

In [12]:
# which clusters are dominant in each year
## Show using heat map
years = np.arange(1979,2019)
dominant_cluster = np.empty((2,40))
for year in years:
    i=year-1979
    if year in july_c1_dominated:
        dominant_cluster[0,i]=0
    if year in july_c3_dominated:
        dominant_cluster[0,i]=1
    if year in july_neither_dominated:
        dominant_cluster[0,i]=2
    if year in august_c1_dominated:
        dominant_cluster[1,i]=0
    if year in august_c3_dominated:
        dominant_cluster[1,i]=1
    if year in august_neither_dominated:
        dominant_cluster[1,i]=2
df = pd.DataFrame(dominant_cluster, index = ["July", "August"], columns = years)
cmap = plt.cm.get_cmap('GnBu_r', 3)
plt.figure()
ax = sns.heatmap(df, annot = False, cmap=cmap, linewidths=.5, linecolor='lightgray', xticklabels=years)#, yticklabels=["July", "August"])
ax.set(xlabel="Years(1979-2018)", ylabel = "Month", title="Monthly Dominant Cluster Classification(SOM)")
colorbar = ax.collections[0].colorbar
colorbar.set_ticks([0.25, 1, 1.75])
colorbar.set_ticklabels(['Cluster 1', 'Cluster 3', 'Neither'])
plt.show()

In [13]:
# Processing the dataset to monthly values for northern hemisphere.

#2t
dataset_2t_zonal = dataset_2t.sel(latitude = slice(90,25)).mean(dim = 'longitude')
print(dataset_2t_zonal)

#wl1
dataset_wl1_zonal = dataset_wl1.sel(latitude = slice(90,25)).mean(dim = 'longitude')
print(dataset_wl1_zonal)

#wl2
dataset_wl2_zonal = dataset_wl2.sel(latitude = slice(90,25)).mean(dim = 'longitude')
print(dataset_wl2_zonal)

<xarray.Dataset>
Dimensions:   (latitude: 87, time: 120)
Coordinates:
  * latitude  (latitude) float32 90.0 89.25 88.5 87.75 ... 27.75 27.0 26.25 25.5
  * time      (time) datetime64[ns] 1979-06-30 1979-07-31 ... 2018-08-31
Data variables:
    t2m       (time, latitude) float32 dask.array<shape=(120, 87), chunksize=(1, 87)>
<xarray.Dataset>
Dimensions:   (latitude: 87, time: 120)
Coordinates:
  * latitude  (latitude) float32 90.0 89.25 88.5 87.75 ... 27.75 27.0 26.25 25.5
  * time      (time) datetime64[ns] 1979-06-30 1979-07-31 ... 2018-08-31
Data variables:
    swvl1     (time, latitude) float32 dask.array<shape=(120, 87), chunksize=(1, 87)>
<xarray.Dataset>
Dimensions:   (latitude: 87, time: 120)
Coordinates:
  * latitude  (latitude) float32 90.0 89.25 88.5 87.75 ... 27.75 27.0 26.25 25.5
  * time      (time) datetime64[ns] 1979-06-30 1979-07-31 ... 2018-08-31
Data variables:
    swvl2     (time, latitude) float32 dask.array<shape=(120, 87), chunksize=(1, 87)>


In [14]:
dataset_2t.t2m[10,:].values

array([[273.9832 , 273.9832 , 273.9832 , ..., 273.9832 , 273.9832 ,
        273.9832 ],
       [273.90677, 273.90933, 273.91217, ..., 273.90497, 273.90564,
        273.90625],
       [273.90903, 273.91205, 273.9149 , ..., 273.91248, 273.9113 ,
        273.9101 ],
       ...,
       [222.4617 , 222.43651, 222.41151, ..., 222.52571, 222.50441,
        222.48302],
       [222.47577, 222.48271, 222.48975, ..., 222.46277, 222.46707,
        222.47122],
       [222.07637, 222.07637, 222.07637, ..., 222.07637, 222.07637,
        222.07637]], dtype=float32)

In [15]:
# And now its monthly data
# And now making the climatology plots

June_t, July_t, August_t = np.zeros((87,480)), np.zeros((87,480)), np.zeros((87,480))
for i in np.arange(0,40):
    June_t += dataset_2t_map.t2m[0+i*3,:,:].values
    July_t += dataset_2t_map.t2m[1+i*3,:,:].values
    August_t += dataset_2t_map.t2m[2+i*3,:,:].values
    
# Calculating the Mean for each month

June_t = June_t/40
July_t = July_t/40
August_t = August_t/40

In [16]:
print(np.amax(June_t),np.amin(June_t))
print(np.amax(July_t),np.amin(July_t))
print(np.amax(August_t),np.amin(August_t))

311.6377899169922 257.17988166809084
313.8183074951172 259.68522186279296
313.04957122802733 256.7878112792969


In [17]:
#Plotting
t_max, t_min, delta = 315, 255, 2.5
colorbar_title = "in K"
june_title = "Climatology map of Surface Temperature for June (1979-2018)"
july_title = "Climatology map of Surface Temperature for July (1979-2018)"
august_title = "Climatology map of Surface Temperature for August (1979-2018)"
draw_on_map([June_t, July_t, August_t],[june_title,july_title,august_title],t_min,t_max,delta,colorbar_title)

In [19]:
# And now anomaly maps for surface temperature

June_t_b, July_t_b, August_t_b = np.zeros((87,480)), np.zeros((87,480)), np.zeros((87,480))
June_t_l, July_t_l, August_t_l = np.zeros((87,480)), np.zeros((87,480)), np.zeros((87,480))

for i in np.arange(0,40):
    if i < 20:
        June_t_b += dataset_2t_map.t2m[0+i*3,:,:].values
        July_t_b += dataset_2t_map.t2m[1+i*3,:,:].values
        August_t_b += dataset_2t_map.t2m[2+i*3,:,:].values
    else:
        June_t_l += dataset_2t_map.t2m[0+i*3,:,:].values
        July_t_l += dataset_2t_map.t2m[1+i*3,:,:].values
        August_t_l += dataset_2t_map.t2m[2+i*3,:,:].values
    
# Calculating the Mean for each month

June_t_b = June_t_b/20
July_t_b = July_t_b/20
August_t_b = August_t_b/20
June_t_l = June_t_l/20
July_t_l = July_t_l/20
August_t_l = August_t_l/20

June_anomaly = June_t_l-June_t_b
July_anomaly = July_t_l-July_t_b
August_anomaly = August_t_l-August_t_b

In [20]:
print(np.amax(June_anomaly),np.amin(June_anomaly))
print(np.amax(July_anomaly),np.amin(July_anomaly))
print(np.amax(August_anomaly),np.amin(August_anomaly))

4.316722106933582 -1.3485214233398892
3.4663208007812614 -1.4609222412109375
3.426206970214878 -1.1901977539062045


In [21]:
#Plotting
t_max, t_min,delta = 4.5, -4.5, 0.25
colorbar_title = "in K"
june_title = "Anomaly map of Surface Temperature for June (1979-2018)"
july_title = "Anomaly map of Surface Temperature for July (1979-2018)"
august_title = "Anomaly map of Surface Temperature for August (1979-2018)"
draw_on_map([June_anomaly, July_anomaly, August_anomaly],[june_title,july_title,august_title],t_min,t_max,delta,colorbar_title,res=4)

In [114]:
# And now trend maps for surface temperature

years = np.arange(0, 40) 
slopes_june_t = np.zeros([87, 480])
intercepts_june_t = np.zeros([87, 480])
slopes_july_t = np.zeros([87, 480])
intercepts_july_t = np.zeros([87, 480])
slopes_august_t = np.zeros([87, 480])
intercepts_august_t = np.zeros([87, 480])

for row in np.arange(87):
    june_values = []
    july_values = []
    august_values = []
    w_array_june = dataset_2t_map.t2m[0:120:3,row,:].values
    w_array_july = dataset_2t_map.t2m[1:120:3,row,:].values
    w_array_august = dataset_2t_map.t2m[2:120:3,row,:].values
    print(row)
    for col in np.arange(480):
        
        
        june_values.append(w_array_june[:,col])
        july_values.append(w_array_july[:,col])
        august_values.append(w_array_august[:,col])
        
        m, b = np.polyfit(years, june_values[col], 1)
        slopes_june_t[row, col] = m
        intercepts_june_t[row, col] = b
        
        m, b = np.polyfit(years, july_values[col], 1)
        slopes_july_t[row, col] = m
        intercepts_july_t[row, col] = b
        
        m, b = np.polyfit(years, august_values[col], 1)
        slopes_august_t[row, col] = m
        intercepts_august_t[row, col] = b

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86


In [116]:
print(np.amax(slopes_june_t*40),np.amin(slopes_june_t*40))
print(np.amax(slopes_july_t*40),np.amin(slopes_july_t*40))
print(np.amax(slopes_august_t*40),np.amin(slopes_august_t*40))
print(w_array_june.shape, len(june_values[2]))

6.938160851569972 -2.229504440336194
7.295101888631575 -4.80020605377132
6.991639557743672 -3.6936264181225766
(40, 480) 40


In [239]:
#Plotting
t_max, t_min,delta = 0.2, -0.2, 0.0125
colorbar_title = "change per annum(in K)"
june_title = "Trend map of Surface Temperature for June (1979-2018)"
july_title = "Trend map of Surface Temperature for July (1979-2018)"
august_title = "Trend map of Surface Temperature for August (1979-2018)"
draw_on_map([slopes_june_t, slopes_july_t, slopes_august_t],[june_title,july_title,august_title],t_min,t_max,delta,colorbar_title,res=4)

In [211]:
# And now July and August composites for Surface Temperature
# Next step is to use seggregated years (c1 dominated, c3 dominated, neither) to find average plots for each variable 
# within these particular years
# And now its monthly data
# Taking into consideration the zonal means

June_jc1t, July_jc1t, August_jc1t = np.zeros((87,480)), np.zeros((87,480)), np.zeros((87,480))
June_jc3t, July_jc3t, August_jc3t = np.zeros((87,480)), np.zeros((87,480)), np.zeros((87,480))
June_jnt, July_jnt, August_jnt = np.zeros((87,480)), np.zeros((87,480)), np.zeros((87,480))

June_ac1t, July_ac1t, August_ac1t = np.zeros((87,480)), np.zeros((87,480)), np.zeros((87,480))
June_ac3t, July_ac3t, August_ac3t = np.zeros((87,480)), np.zeros((87,480)), np.zeros((87,480))
June_ant, July_ant, August_ant = np.zeros((87,480)), np.zeros((87,480)), np.zeros((87,480))

for i in np.arange(0,40):
    year = 1979+i
    print(year)
    if year in july_c1_dominated:
        #June_jc1t += dataset_2t_map.t2m[0+i*3,:,:].values
        July_jc1t += dataset_2t_map.t2m[1+i*3,:,:].values
        August_jc1t += dataset_2t_map.t2m[2+i*3,:,:].values
    elif year in july_c3_dominated:
        #June_jc3t += dataset_2t_map.t2m[0+i*3,:,:].values
        July_jc3t += dataset_2t_map.t2m[1+i*3,:,:].values
        August_jc3t += dataset_2t_map.t2m[2+i*3,:,:].values
    elif year in july_neither_dominated:
        #June_jnt += dataset_2t_map.t2m[0+i*3,:,:].values
        July_jnt += dataset_2t_map.t2m[1+i*3,:,:].values
        August_jnt += dataset_2t_map.t2m[2+i*3,:,:].values
    
    if year in august_c1_dominated:
        #June_ac1t += dataset_2t_map.t2m[0+i*3,:,:].values
        July_ac1t += dataset_2t_map.t2m[1+i*3,:,:].values
        August_ac1t += dataset_2t_map.t2m[2+i*3,:,:].values
    elif year in august_c3_dominated:
        #June_ac3t += dataset_2t_map.t2m[0+i*3,:,:].values
        July_ac3t += dataset_2t_map.t2m[1+i*3,:,:].values
        August_ac3t += dataset_2t_map.t2m[2+i*3,:,:].values
    elif year in august_neither_dominated:
        #June_ant += dataset_2t_map.t2m[0+i*3,:,:].values
        July_ant += dataset_2t_map.t2m[1+i*3,:,:].values
        August_ant += dataset_2t_map.t2m[2+i*3,:,:].values
    
    
# Calculating the Mean for each month

#June_jc1t = June_jc1t/len(july_c1_dominated)
July_jc1t, August_jc1t = July_jc1t/len(july_c1_dominated), August_jc1t/len(july_c1_dominated)
#June_jc3t = June_jc3t/len(july_c3_dominated),
July_jc3t, August_jc3t = July_jc3t/len(july_c3_dominated), August_jc3t/len(july_c3_dominated)
#June_jnt = June_jnt/len(july_neither_dominated) 
July_jnt, August_jnt = July_jnt/len(july_neither_dominated), August_jnt/len(july_neither_dominated)

#June_ac1t = June_ac1t/len(august_c1_dominated),
July_ac1t, August_ac1t = July_ac1t/len(august_c1_dominated), August_ac1t/len(august_c1_dominated)
#June_ac3t = June_ac3t/len(august_c3_dominated),
July_ac3t, August_ac3t = July_ac3t/len(august_c3_dominated), August_ac3t/len(august_c3_dominated)
#June_ant = June_ant/len(august_neither_dominated),
July_ant, August_ant = July_ant/len(august_neither_dominated), August_ant/len(august_neither_dominated)

1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018


In [212]:
print("July-C1")
print(np.amax(July_jc1t-July_t),np.amin(July_jc1t-July_t))
print(np.amax(August_jc1t-August_t),np.amin(August_jc1t-August_t))
print("July-C3")
print(np.amax(July_jc3t-July_t),np.amin(July_jc3t-July_t))
print(np.amax(August_jc3t-August_t),np.amin(August_jc3t-August_t))
print("July-Neither")
print(np.amax(July_jnt-July_t),np.amin(July_jnt-July_t))
print(np.amax(August_jnt-August_t),np.amin(August_jnt-August_t))

print("August-C1")
print(np.amax(July_ac1t-July_t),np.amin(July_ac1t-July_t))
print(np.amax(August_ac1t-August_t),np.amin(August_ac1t-August_t))
print("August-C3")
print(np.amax(July_ac3t-July_t),np.amin(July_ac3t-July_t))
print(np.amax(August_ac3t-August_t),np.amin(August_ac3t-August_t))
print("August-Neither")
print(np.amax(July_ant-July_t),np.amin(July_ant-July_t))
print(np.amax(August_ant-August_t),np.amin(August_ant-August_t))

July-C1
1.0506394606370009 -1.805611126239512
0.9166560246393942 -1.3097504248985388
July-C3
2.284002568171559 -1.0329100388747179
1.312554872953001 -0.8299689072828755
July-Neither
0.9270827157156418 -1.0752693176269759
0.9285476684570426 -0.9778508867536289
August-C1
1.2472498380220713 -1.3833395737868273
1.0520951491135975 -1.620970799372742
August-C3
1.5626657339242342 -0.8855901864858993
2.0463171738844608 -0.7969990950364263
August-Neither
1.270820944649813 -0.9422958374023551
1.096273149762851 -0.8499606541224694


In [220]:
#Plotting parameters
t_max, t_min, delta = 2.5, -2.5, 0.25
colorbar_title = "in K"

In [221]:
# more Plotting parameters
jul_aug_title = "Surface Temperature for July and August relative to Average Surface Temperature(1979-2018)"
july_jc1 = "July(C1-July-dominated)"
august_jc1 = "August(C1-July-dominated)"

july_jc3 = "July(C3-July-dominated)"
august_jc3 = "August(C3-July-dominated)"

july_jn = "July(Neither-July-dominated)"
august_jn = "August(Neither-July-dominated)"

july_ac1 = "July(C1-August-dominated)"
august_ac1 = "August(C1-August-dominated)"

july_ac3 = "July(C3-August-dominated)"
august_ac3 = "August(C3-August-dominated)"

july_an = "July(Neither-August-dominated)"
august_an = "August(Neither-August-dominated)"

#Plotting
#C1-Dominant
draw_on_map([July_jc1t-July_t, August_jc1t-August_t],[july_jc1,august_jc1],t_min,t_max,delta,colorbar_title,split=2,sup_title=jul_aug_title)
draw_on_map([July_ac1t-July_t, August_ac1t-August_t],[july_ac1,august_ac1],t_min,t_max,delta,colorbar_title,split=2,sup_title=jul_aug_title)

#C3-Dominant
draw_on_map([July_jc3t-July_t, August_jc3t-August_t],[july_jc3,august_jc3],t_min,t_max,delta,colorbar_title,split=2,sup_title=jul_aug_title)
draw_on_map([July_ac3t-July_t, August_ac3t-August_t],[july_ac3,august_ac3],t_min,t_max,delta,colorbar_title,split=2,sup_title=jul_aug_title)

#Neither-Dominant
draw_on_map([July_jnt-July_t, August_jnt-August_t],[july_jn,august_jn],t_min,t_max,delta,colorbar_title,split=2,sup_title=jul_aug_title)
draw_on_map([July_ant-July_t, August_ant-August_t],[july_an,august_an],t_min,t_max,delta,colorbar_title,split=2,sup_title=jul_aug_title)


In [22]:
# Soil Water Volumetric Level - 1
#Climatology
June_sw1, July_sw1, August_sw1 = np.zeros((87,480)), np.zeros((87,480)), np.zeros((87,480))
for i in np.arange(0,40):
    June_sw1 += dataset_wl1_map.swvl1[0+i*3,:,:].values
    July_sw1 += dataset_wl1_map.swvl1[1+i*3,:,:].values
    August_sw1 += dataset_wl1_map.swvl1[2+i*3,:,:].values
    
# Calculating the Mean for each month

June_sw1 = June_sw1/40
July_sw1 = July_sw1/40
August_sw1 = August_sw1/40

In [23]:
# Determining the range of colorbar
print(np.amax(June_sw1),np.amin(June_sw1))
print(np.amax(July_sw1),np.amin(July_sw1))
print(np.amax(August_sw1),np.amin(August_sw1))

0.7426930114626884 -1.3135373850836628e-07
0.7445847406983376 -1.1489035998124564e-06
0.7441683605313301 -2.0016825198476694e-07


In [250]:
#Plotting

w_max, w_min, delta = 1, 0, 0.1
colorbar_title = "in m^3 m^-3"  # unit described on era5 m^3m^-3
june_title = "Climatology map of Soil Water Volumetric Level 1 for June (1979-2018)"
july_title = "Climatology map of Soil Water Volumetric Level 1 for July (1979-2018)"
august_title = "Climatology map of Soil Water Volumetric Level 1 for August (1979-2018)"
draw_on_map([June_sw1, July_sw1, August_sw1],[june_title,july_title,august_title],w_min,w_max,delta, colorbar_title,res=2, color="Blues")

In [251]:
# Soil Water Volumetric Level - 2
#Climatology
June_sw2, July_sw2, August_sw2 = np.zeros((87,480)), np.zeros((87,480)), np.zeros((87,480))
for i in np.arange(0,40):
    June_sw2 += dataset_wl2_map.swvl2[0+i*3,:,:].values
    July_sw2 += dataset_wl2_map.swvl2[1+i*3,:,:].values
    August_sw2 += dataset_wl2_map.swvl2[2+i*3,:,:].values
    
# Calculating the Mean for each month

June_sw2 = June_sw2/40
July_sw2 = July_sw2/40
August_sw2 = August_sw2/40

In [252]:
# Determining the range of colorbar
print(np.amax(June_sw2),np.amin(June_sw2))
print(np.amax(July_sw2),np.amin(July_sw2))
print(np.amax(August_sw2),np.amin(August_sw2))

0.7436719045042992 1.5522044970184278e-08
0.7452790081501007 -6.284924811694736e-08
0.7455668047070503 9.388933244203202e-08


In [254]:
#Plotting

w_max, w_min, delta = 1, 0, 0.05
colorbar_title = "in m^3 m^-3"  # unit described on era5 m^3m^-3
june_title = "Climatology map of Soil Water Volumetric Level 2 for June (1979-2018)"
july_title = "Climatology map of Soil Water Volumetric Level 2 for July (1979-2018)"
august_title = "Climatology map of Soil Water Volumetric Level 2 for August (1979-2018)"
draw_on_map([June_sw2, July_sw2, August_sw2],[june_title,july_title,august_title],w_min,w_max,delta,colorbar_title,res=4,color="Blues")

In [43]:
#Trend Maps


years = np.arange(0, 40) 
slopes_june_sw1 = np.zeros([87, 480])
intercepts_june_sw1 = np.zeros([87, 480])
slopes_july_sw1 = np.zeros([87, 480])
intercepts_july_sw1 = np.zeros([87, 480])
slopes_august_sw1 = np.zeros([87, 480])
intercepts_august_sw1 = np.zeros([87, 480])

for row in np.arange(87):
    june_values = []
    july_values = []
    august_values = []
    w_array_june = dataset_wl1_map.swvl1[0:120:3,row,:].values
    w_array_july = dataset_wl1_map.swvl1[1:120:3,row,:].values
    w_array_august = dataset_wl1_map.swvl1[2:120:3,row,:].values
    print(row)
    for col in np.arange(480):
        
        
        june_values.append(w_array_june[:,col])
        july_values.append(w_array_july[:,col])
        august_values.append(w_array_august[:,col])
        
        m, b = np.polyfit(years, june_values[col], 1)
        slopes_june_sw1[row, col] = m
        intercepts_june_sw1[row, col] = b
        
        m, b = np.polyfit(years, july_values[col], 1)
        slopes_july_sw1[row, col] = m
        intercepts_july_sw1[row, col] = b
        
        m, b = np.polyfit(years, august_values[col], 1)
        slopes_august_sw1[row, col] = m
        intercepts_august_sw1[row, col] = b

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86


In [44]:
# Determining the range of colorbar        
print(np.amax(slopes_june_sw1),np.amin(slopes_june_sw1))
print(np.amax(slopes_july_sw1),np.amin(slopes_july_sw1))
print(np.amax(slopes_august_sw1),np.amin(slopes_august_sw1))
print(w_array_june.shape, len(june_values[2]))

0.010040568841925217 -0.005352150249045065
0.010043490868197554 -0.005411522030327147
0.010081174000450378 -0.005331975547334707
(40, 480) 40


In [51]:
#Plotting
w_max, w_min,delta = 0.0105, -0.0105, 0.001
colorbar_title = "change per annum(in m^3 m^-3)"
june_title = "Trend map of Soil Water Volumetric Layer 1 for June (1979-2018)"
july_title = "Trend map of Soil Water Volumetric Layer 1 for July (1979-2018)"
august_title = "Trend map of Soil Water Volumetric Layer 1 for August (1979-2018)"
draw_on_map([slopes_june_sw1, slopes_july_sw1, slopes_august_sw1],[june_title,july_title,august_title],w_min,w_max,delta,colorbar_title,res=3,color="seismic")

In [ ]:
#Trend Maps


years = np.arange(0, 40) 
slopes_june_sw2 = np.zeros([87, 480])
intercepts_june_sw2 = np.zeros([87, 480])
slopes_july_sw2 = np.zeros([87, 480])
intercepts_july_sw2 = np.zeros([87, 480])
slopes_august_sw2 = np.zeros([87, 480])
intercepts_august_sw2 = np.zeros([87, 480])

for row in np.arange(87):
    june_values = []
    july_values = []
    august_values = []
    w_array_june = dataset_wl2_map.swvl2[0:120:3,row,:].values
    w_array_july = dataset_wl2_map.swvl2[1:120:3,row,:].values
    w_array_august = dataset_wl2_map.swvl2[2:120:3,row,:].values
    print(row)
    for col in np.arange(480):
        
        
        june_values.append(w_array_june[:,col])
        july_values.append(w_array_july[:,col])
        august_values.append(w_array_august[:,col])
        
        m, b = np.polyfit(years, june_values[col], 1)
        slopes_june_sw2[row, col] = m
        intercepts_june_sw2[row, col] = b
        
        m, b = np.polyfit(years, july_values[col], 1)
        slopes_july_sw2[row, col] = m
        intercepts_july_sw2[row, col] = b
        
        m, b = np.polyfit(years, august_values[col], 1)
        slopes_august_sw2[row, col] = m
        intercepts_august_sw2[row, col] = b

In [311]:
        
print(np.amax(slopes_june_sw2),np.amin(slopes_june_sw2))
print(np.amax(slopes_july_sw2),np.amin(slopes_july_sw2))
print(np.amax(slopes_august_sw2),np.amin(slopes_august_sw2))
print(w_array_june.shape, len(june_values[2]))

0.007927972567363207 -0.00883575315495436
0.007944106075889744 -0.008783954950535383
0.007940190097601848 -0.00882581975639406
(40, 480) 40


In [2]:
#Plotting
w_max, w_min,delta = 0.0105, -0.0105, 0.002
colorbar_title = "change per annum(in m^3 m^-3)"
june_title = "Trend map of Soil Water Volumetric Layer 2 for June (1979-2018)"
july_title = "Trend map of Soil Water Volumetric Layer 2 for July (1979-2018)"
august_title = "Trend map of Soil Water Volumetric Layer 2 for August (1979-2018)"
draw_on_map([slopes_june_sw2, slopes_july_sw2, slopes_august_sw2],[june_title,july_title,august_title],w_min,w_max,delta,colorbar_title,res=1)

NameError: name 'draw_on_map' is not defined

In [24]:
#Anomaly Maps

June_sw1_b, July_sw1_b, August_sw1_b = np.zeros((87,480)), np.zeros((87,480)), np.zeros((87,480))
June_sw1_l, July_sw1_l, August_sw1_l = np.zeros((87,480)), np.zeros((87,480)), np.zeros((87,480))

for i in np.arange(0,40):
    if i < 20:
        June_sw1_b += dataset_wl1_map.swvl1[0+i*3,:,:].values
        July_sw1_b += dataset_wl1_map.swvl1[1+i*3,:,:].values
        August_sw1_b += dataset_wl1_map.swvl1[2+i*3,:,:].values
    else:
        June_sw1_l += dataset_wl1_map.swvl1[0+i*3,:,:].values
        July_sw1_l += dataset_wl1_map.swvl1[1+i*3,:,:].values
        August_sw1_l += dataset_wl1_map.swvl1[2+i*3,:,:].values
    
# Calculating the Mean for each month

June_sw1_b = June_sw1_b/20
July_sw1_b = July_sw1_b/20
August_sw1_b = August_sw1_b/20
June_sw1_l = June_sw1_l/20
July_sw1_l = July_sw1_l/20
August_sw1_l = August_sw1_l/20

June_anomaly = June_sw1_l-June_sw1_b
July_anomaly = July_sw1_l-July_sw1_b
August_anomaly = August_sw1_l-August_sw1_b

In [25]:
# Determining the range
print(np.amax(June_anomaly),np.amin(June_anomaly))
print(np.amax(July_anomaly),np.amin(July_anomaly))
print(np.amax(August_anomaly),np.amin(August_anomaly))

0.2311773359465093 -0.16231596544384955
0.2311641544794604 -0.16373315230011937
0.23186206576684162 -0.1605233855545521


In [42]:
#Plotting
w_max, w_min, delta = 0.25, -0.25,0.02
colorbar_title = "in m^3 m^-3"
june_title = "Anomaly map of Soil Water Volumetric Layer 1 for June (1979-2018)"
july_title = "Anomaly map of Soil Water Volumetric Layer 1 for July (1979-2018)"
august_title = "Anomaly map of Soil Water Volumetric Layer 1 for August (1979-2018)"
draw_on_map([June_anomaly, July_anomaly, August_anomaly],[june_title,july_title,august_title],w_min,w_max,delta,colorbar_title,res=5)

In [271]:
#Anomaly Maps

June_sw2_b, July_sw2_b, August_sw2_b = np.zeros((87,480)), np.zeros((87,480)), np.zeros((87,480))
June_sw2_l, July_sw2_l, August_sw2_l = np.zeros((87,480)), np.zeros((87,480)), np.zeros((87,480))

for i in np.arange(0,40):
    if i < 20:
        June_sw2_b += dataset_wl2_map.swvl2[0+i*3,:,:].values
        July_sw2_b += dataset_wl2_map.swvl2[1+i*3,:,:].values
        August_sw2_b += dataset_wl2_map.swvl2[2+i*3,:,:].values
    else:
        June_sw2_l += dataset_wl2_map.swvl2[0+i*3,:,:].values
        July_sw2_l += dataset_wl2_map.swvl2[1+i*3,:,:].values
        August_sw2_l += dataset_wl2_map.swvl2[2+i*3,:,:].values
    
# Calculating the Mean for each month

June_sw2_b = June_sw2_b/20
July_sw2_b = July_sw2_b/20
August_sw2_b = August_sw2_b/20
June_sw2_l = June_sw2_l/20
July_sw2_l = July_sw2_l/20
August_sw2_l = August_sw2_l/20

June_anomaly = June_sw2_l-June_sw2_b
July_anomaly = July_sw2_l-July_sw2_b
August_anomaly = August_sw2_l-August_sw2_b

In [334]:
#Determing the range of colorbar
print(np.amax(June_anomaly),np.amin(June_anomaly))
print(np.amax(July_anomaly),np.amin(July_anomaly))
print(np.amax(August_anomaly),np.amin(August_anomaly))

0.15076863687484288 -0.2137301043141633
0.1512026506866574 -0.21121124642781253
0.15125703928812903 -0.21495926522295558


In [347]:
#Plotting
w_max, w_min, delta = 0.225, -0.225,0.03
colorbar_title = "in m^3 m^-3"
june_title = "Anomaly map of Soil Water Volumetric Layer 2 for June (1979-2018)"
july_title = "Anomaly map of Soil Water Volumetric Layer 2 for July (1979-2018)"
august_title = "Anomaly map of Soil Water Volumetric Layer 2 for August (1979-2018)"
draw_on_map([June_anomaly, July_anomaly, August_anomaly],[june_title,july_title,august_title],w_min,w_max,delta,colorbar_title,res=1)

In [275]:
#Composite Maps 


June_jc1sw1, July_jc1sw1, August_jc1sw1 = np.zeros((87,480)), np.zeros((87,480)), np.zeros((87,480))
June_jc3sw1, July_jc3sw1, August_jc3sw1 = np.zeros((87,480)), np.zeros((87,480)), np.zeros((87,480))
June_jnsw1, July_jnsw1, August_jnsw1 = np.zeros((87,480)), np.zeros((87,480)), np.zeros((87,480))

June_ac1sw1, July_ac1sw1, August_ac1sw1 = np.zeros((87,480)), np.zeros((87,480)), np.zeros((87,480))
June_ac3sw1, July_ac3sw1, August_ac3sw1 = np.zeros((87,480)), np.zeros((87,480)), np.zeros((87,480))
June_answ1, July_answ1, August_answ1 = np.zeros((87,480)), np.zeros((87,480)), np.zeros((87,480))

for i in np.arange(0,40):
    year = 1979+i
    print(year)
    if year in july_c1_dominated:
        #June_jc1t += dataset_2t_map.t2m[0+i*3,:,:].values
        July_jc1sw1 += dataset_wl1_map.swvl1[1+i*3,:,:].values
        August_jc1sw1 += dataset_wl1_map.swvl1[2+i*3,:,:].values
    elif year in july_c3_dominated:
        #June_jc3t += dataset_2t_map.t2m[0+i*3,:,:].values
        July_jc3sw1 += dataset_wl1_map.swvl1[1+i*3,:,:].values
        August_jc3sw1 += dataset_wl1_map.swvl1[2+i*3,:,:].values
    elif year in july_neither_dominated:
        #June_jnt += dataset_2t_map.t2m[0+i*3,:,:].values
        July_jnsw1 += dataset_wl1_map.swvl1[1+i*3,:,:].values
        August_jnsw1 += dataset_wl1_map.swvl1[2+i*3,:,:].values
    
    if year in august_c1_dominated:
        #June_ac1t += dataset_2t_map.t2m[0+i*3,:,:].values
        July_ac1sw1 += dataset_wl1_map.swvl1[1+i*3,:,:].values
        August_ac1sw1 += dataset_wl1_map.swvl1[2+i*3,:,:].values
    elif year in august_c3_dominated:
        #June_ac3t += dataset_2t_map.t2m[0+i*3,:,:].values
        July_ac3sw1 += dataset_wl1_map.swvl1[1+i*3,:,:].values
        August_ac3sw1 += dataset_wl1_map.swvl1[2+i*3,:,:].values
    elif year in august_neither_dominated:
        #June_ant += dataset_2t_map.t2m[0+i*3,:,:].values
        July_answ1 += dataset_wl1_map.swvl1[1+i*3,:,:].values
        August_answ1 += dataset_wl1_map.swvl1[2+i*3,:,:].values
    
    
# Calculating the Mean for each month

#June_jc1t = June_jc1t/len(july_c1_dominated)
July_jc1sw1, August_jc1sw1 = July_jc1sw1/len(july_c1_dominated), August_jc1sw1/len(july_c1_dominated)
#June_jc3t = June_jc3t/len(july_c3_dominated),
July_jc3sw1, August_jc3sw1 = July_jc3sw1/len(july_c3_dominated), August_jc3sw1/len(july_c3_dominated)
#June_jnt = June_jnt/len(july_neither_dominated) 
July_jnsw1, August_jnsw1 = July_jnsw1/len(july_neither_dominated), August_jnsw1/len(july_neither_dominated)

#June_ac1t = June_ac1t/len(august_c1_dominated),
July_ac1sw1, August_ac1sw1 = July_ac1sw1/len(august_c1_dominated), August_ac1sw1/len(august_c1_dominated)
#June_ac3t = June_ac3t/len(august_c3_dominated),
July_ac3sw1, August_ac3sw1 = July_ac3sw1/len(august_c3_dominated), August_ac3sw1/len(august_c3_dominated)
#June_ant = June_ant/len(august_neither_dominated),
July_answ1, August_answ1 = July_answ1/len(august_neither_dominated), August_answ1/len(august_neither_dominated)

1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018


In [276]:
#Determining the range of colorbar

print("July-C1")
print(np.amax(July_jc1sw1-July_sw1),np.amin(July_jc1sw1-July_sw1))
print(np.amax(August_jc1sw1-August_sw1),np.amin(August_jc1sw1-August_sw1))
print("July-C3")
print(np.amax(July_jc3sw1-July_sw1),np.amin(July_jc3sw1-July_sw1))
print(np.amax(August_jc3sw1-August_sw1),np.amin(August_jc3sw1-August_sw1))
print("July-Neither")
print(np.amax(July_jnsw1-July_sw1),np.amin(July_jnsw1-July_sw1))
print(np.amax(August_jnsw1-August_sw1),np.amin(August_jnsw1-August_sw1))

print("August-C1")
print(np.amax(July_ac1sw1-July_sw1),np.amin(July_ac1sw1-July_sw1))
print(np.amax(August_ac1sw1-August_sw1),np.amin(August_ac1sw1-August_sw1))
print("August-C3")
print(np.amax(July_ac3sw1-July_sw1),np.amin(July_ac3sw1-July_sw1))
print(np.amax(August_ac3sw1-August_sw1),np.amin(August_ac3sw1-August_sw1))
print("August-Neither")
print(np.amax(July_answ1-July_sw1),np.amin(July_answ1-July_sw1))
print(np.amax(August_answ1-August_sw1),np.amin(August_answ1-August_sw1))

July-C1
0.055492391530424345 -0.04670120463348354
0.06952419657833303 -0.040107408165931724
July-C3
0.03604665498894 -0.07851033627700349
0.041712444562178375 -0.07377563843933435
July-Neither
0.048587515431323225 -0.05462354719638829
0.043888088635035905 -0.0659502656065992
August-C1
0.05215451570514301 -0.043005805176038
0.06963200438767672 -0.047955665880670906
August-C3
0.03519280614416935 -0.05570196754370743
0.04407642087731241 -0.08324939554127361
August-Neither
0.05002995897084475 -0.055857240308017454
0.03870943382914577 -0.0555581530732013


In [301]:
#Plotting parameters
w_max, w_min, delta = 0.085, -0.085, 0.01
colorbar_title = "in m^3 m^-3"

# more Plotting parameters
jul_aug_title = "Soil Water Volumetric Layer 1 for July and August relative to Average Soil Water Volumetric Layer 1(1979-2018)"
july_jc1 = "July(C1-July-dominated)"
august_jc1 = "August(C1-July-dominated)"

july_jc3 = "July(C3-July-dominated)"
august_jc3 = "August(C3-July-dominated)"

july_jn = "July(Neither-July-dominated)"
august_jn = "August(Neither-July-dominated)"

july_ac1 = "July(C1-August-dominated)"
august_ac1 = "August(C1-August-dominated)"

july_ac3 = "July(C3-August-dominated)"
august_ac3 = "August(C3-August-dominated)"

july_an = "July(Neither-August-dominated)"
august_an = "August(Neither-August-dominated)"

#Plotting
#C1-Dominant
draw_on_map([July_jc1sw1-July_sw1, August_jc1sw1-August_sw1],[july_jc1,august_jc1],w_min,w_max,delta,colorbar_title,split=2,sup_title=jul_aug_title,res=3)
draw_on_map([July_ac1sw1-July_sw1, August_ac1sw1-August_sw1],[july_ac1,august_ac1],w_min,w_max,delta,colorbar_title,split=2,sup_title=jul_aug_title,res=3)

#C3-Dominant
draw_on_map([July_jc3sw1-July_sw1, August_jc3sw1-August_sw1],[july_jc3,august_jc3],w_min,w_max,delta,colorbar_title,split=2,sup_title=jul_aug_title,res=3)
draw_on_map([July_ac3sw1-July_sw1, August_ac3sw1-August_sw1],[july_ac3,august_ac3],w_min,w_max,delta,colorbar_title,split=2,sup_title=jul_aug_title,res=3)

#Neither-Dominant
draw_on_map([July_jnsw1-July_sw1, August_jnsw1-August_sw1],[july_jn,august_jn],w_min,w_max,delta,colorbar_title,split=2,sup_title=jul_aug_title,res=3)
draw_on_map([July_answ1-July_sw1, August_answ1-August_sw1],[july_an,august_an],w_min,w_max,delta,colorbar_title,split=2,sup_title=jul_aug_title,res=3)

In [302]:
#Composite Maps 


June_jc1sw2, July_jc1sw2, August_jc1sw2 = np.zeros((87,480)), np.zeros((87,480)), np.zeros((87,480))
June_jc3sw2, July_jc3sw2, August_jc3sw2 = np.zeros((87,480)), np.zeros((87,480)), np.zeros((87,480))
June_jnsw2, July_jnsw2, August_jnsw2 = np.zeros((87,480)), np.zeros((87,480)), np.zeros((87,480))

June_ac1sw2, July_ac1sw2, August_ac1sw2 = np.zeros((87,480)), np.zeros((87,480)), np.zeros((87,480))
June_ac3sw2, July_ac3sw2, August_ac3sw2 = np.zeros((87,480)), np.zeros((87,480)), np.zeros((87,480))
June_answ2, July_answ2, August_answ2 = np.zeros((87,480)), np.zeros((87,480)), np.zeros((87,480))

for i in np.arange(0,40):
    year = 1979+i
    print(year)
    if year in july_c1_dominated:
        #June_jc1t += dataset_2t_map.t2m[0+i*3,:,:].values
        July_jc1sw2 += dataset_wl2_map.swvl2[1+i*3,:,:].values
        August_jc1sw2 += dataset_wl2_map.swvl2[i*3,:,:].values
    elif year in july_c3_dominated:
        #June_jc3t += dataset_2t_map.t2m[0+i*3,:,:].values
        July_jc3sw2 += dataset_wl2_map.swvl2[1+i*3,:,:].values
        August_jc3sw2 += dataset_wl2_map.swvl2[2+i*3,:,:].values
    elif year in july_neither_dominated:
        #June_jnt += dataset_2t_map.t2m[0+i*3,:,:].values
        July_jnsw2 += dataset_wl2_map.swvl2[1+i*3,:,:].values
        August_jnsw2 += dataset_wl2_map.swvl2[2+i*3,:,:].values
    
    if year in august_c1_dominated:
        #June_ac1t += dataset_2t_map.t2m[0+i*3,:,:].values
        July_ac1sw2 += dataset_wl2_map.swvl2[1+i*3,:,:].values
        August_ac1sw2 += dataset_wl2_map.swvl2[2+i*3,:,:].values
    elif year in august_c3_dominated:
        #June_ac3t += dataset_2t_map.t2m[0+i*3,:,:].values
        July_ac3sw2 += dataset_wl2_map.swvl2[1+i*3,:,:].values
        August_ac3sw2 += dataset_wl2_map.swvl2[2+i*3,:,:].values
    elif year in august_neither_dominated:
        #June_ant += dataset_2t_map.t2m[0+i*3,:,:].values
        July_answ2 += dataset_wl2_map.swvl2[1+i*3,:,:].values
        August_answ2 += dataset_wl2_map.swvl2[2+i*3,:,:].values
    
    
# Calculating the Mean for each month

#June_jc1t = June_jc1t/len(july_c1_dominated)
July_jc1sw2, August_jc1sw2 = July_jc1sw2/len(july_c1_dominated), August_jc1sw2/len(july_c1_dominated)
#June_jc3t = June_jc3t/len(july_c3_dominated),
July_jc3sw2, August_jc3sw2 = July_jc3sw2/len(july_c3_dominated), August_jc3sw2/len(july_c3_dominated)
#June_jnt = June_jnt/len(july_neither_dominated) 
July_jnsw2, August_jnsw2 = July_jnsw2/len(july_neither_dominated), August_jnsw2/len(july_neither_dominated)

#June_ac1t = June_ac1t/len(august_c1_dominated),
July_ac1sw2, August_ac1sw2 = July_ac1sw2/len(august_c1_dominated), August_ac1sw2/len(august_c1_dominated)
#June_ac3t = June_ac3t/len(august_c3_dominated),
July_ac3sw2, August_ac3sw2 = July_ac3sw2/len(august_c3_dominated), August_ac3sw2/len(august_c3_dominated)
#June_ant = June_ant/len(august_neither_dominated),
July_answ2, August_answ2 = July_answ2/len(august_neither_dominated), August_answ2/len(august_neither_dominated)

1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018


In [303]:
# Determining the range of colorbar
print("July-C1")
print(np.amax(July_jc1sw2-July_sw2),np.amin(July_jc1sw2-July_sw2))
print(np.amax(August_jc1sw2-August_sw2),np.amin(August_jc1sw2-August_sw2))
print("July-C3")
print(np.amax(July_jc3sw2-July_sw2),np.amin(July_jc3sw2-July_sw2))
print(np.amax(August_jc3sw2-August_sw2),np.amin(August_jc3sw2-August_sw2))
print("July-Neither")
print(np.amax(July_jnsw2-July_sw2),np.amin(July_jnsw2-July_sw2))
print(np.amax(August_jnsw2-August_sw2),np.amin(August_jnsw2-August_sw2))

print("August-C1")
print(np.amax(July_ac1sw2-July_sw2),np.amin(July_ac1sw2-July_sw2))
print(np.amax(August_ac1sw2-August_sw2),np.amin(August_ac1sw2-August_sw2))
print("August-C3")
print(np.amax(July_ac3sw2-July_sw2),np.amin(July_ac3sw2-July_sw2))
print(np.amax(August_ac3sw2-August_sw2),np.amin(August_ac3sw2-August_sw2))
print("August-Neither")
print(np.amax(July_answ2-July_sw2),np.amin(July_answ2-July_sw2))
print(np.amax(August_answ2-August_sw2),np.amin(August_answ2-August_sw2))

July-C1
0.04473935686624969 -0.03979931720174279
0.14984541601286483 -0.20776576540217953
July-C3
0.03962001544065202 -0.04269659304274964
0.03811334676650162 -0.053172674717811425
July-Neither
0.03435950055718423 -0.035500350195382335
0.03453097779835973 -0.044582798125754525
August-C1
0.03716453961454902 -0.04242787233625467
0.04242822224130999 -0.04665703174586483
August-C3
0.03423393994879609 -0.03928338538747854
0.0347031084390787 -0.05039130546725712
August-Neither
0.03468666061603472 -0.052256423545238545
0.04094987676611972 -0.05208701734314116


In [307]:
#Plotting parameters
w_max, w_min, delta = 0.21, -0.21, 0.02
colorbar_title = "in m^3 m^-3"

# more Plotting parameters
jul_aug_title = "Soil Water Volumetric Layer 2 for July and August relative to Average Soil Water Volumetric Layer 2(1979-2018)"
july_jc1 = "July(C1-July-dominated)"
august_jc1 = "August(C1-July-dominated)"

july_jc3 = "July(C3-July-dominated)"
august_jc3 = "August(C3-July-dominated)"

july_jn = "July(Neither-July-dominated)"
august_jn = "August(Neither-July-dominated)"

july_ac1 = "July(C1-August-dominated)"
august_ac1 = "August(C1-August-dominated)"

july_ac3 = "July(C3-August-dominated)"
august_ac3 = "August(C3-August-dominated)"

july_an = "July(Neither-August-dominated)"
august_an = "August(Neither-August-dominated)"

#Plotting
#C1-Dominant
draw_on_map([July_jc1sw2-July_sw2, August_jc1sw2-August_sw2],[july_jc1,august_jc1],w_min,w_max,delta,colorbar_title,split=2,sup_title=jul_aug_title,res=3)
draw_on_map([July_ac1sw2-July_sw2, August_ac1sw2-August_sw2],[july_ac1,august_ac1],w_min,w_max,delta,colorbar_title,split=2,sup_title=jul_aug_title,res=3)

#C3-Dominant
draw_on_map([July_jc3sw2-July_sw2, August_jc3sw2-August_sw2],[july_jc3,august_jc3],w_min,w_max,delta,colorbar_title,split=2,sup_title=jul_aug_title,res=3)
draw_on_map([July_ac3sw2-July_sw2, August_ac3sw2-August_sw2],[july_ac3,august_ac3],w_min,w_max,delta,colorbar_title,split=2,sup_title=jul_aug_title,res=3)

#Neither-Dominant
draw_on_map([July_jnsw2-July_sw2, August_jnsw2-August_sw2],[july_jn,august_jn],w_min,w_max,delta,colorbar_title,split=2,sup_title=jul_aug_title,res=3)
draw_on_map([July_a]\            \sw2-July_sw2, August_answ2-August_sw2],[july_an,august_an],w_min,w_max,delta,colorbar_title,split=2,sup_title=jul_aug_title,res=3)


In [16]:
# Zonal Surface Temperature Mosaic for July, and August

def temperature_mosaic(month):
    """
    -June, 1-July, 2-August\[]
    
    """
    if month == 'July':
        month_val = 1
         
    elif month == 'June':
        month_val = 0
        
    elif month == 'August':
        month_val = 2
        
    else:
        print("Do not have data for this particular month\n")
        return
    temperatures = np.empty((87,40))
    for i in np.arange(40):
        temperatures[:,i] = dataset_2t.t2m[month_val+i*3,:]
    latitudes = dataset_2t.t2m.latitude.values
    years = np.arange(1979, 2019)
    df_temperature = pd.DataFrame(temperatures, index = latitudes, columns = years)
    cmap = plt.cm.get_cmap('seismic', 30)
    plt.figure()
    ax = sns.heatmap(df_temperature, annot = False, cmap=cmap, linewidths=.01, linecolor='lightgray')
    ax.set(xlabel="Years(1979-2018)", ylabel="Latitude", title="Zonal Surface Temperature %s"%(month))
    # Manually specify colorbar labelling after it's been generated
    colorbar = ax.collections[0].colorbar
    #colorbar.set_ticks([0.25, 1, 1.75])
    #colorbar.set_ticklabels(['Cluster 1', 'Cluster 2', 'Cluster 3'])
    plt.show()
    return df_temperature

def temperature_deviations_mosaic(month):
    """
    0-June, 1-July, 2-August
    """
    if month == 'July':
        month_val = 1
        month_t = July_t
    elif month == 'June':
        month_val = 0
        month_t = June_t
    elif month == 'August':
        month_val = 2
        month_t = August_t
    else:
        print("Do not have data for this particular month\n")
        return
    temperatures = np.empty((87,40))
    for i in np.arange(40):
        temperatures[:,i] = np.subtract(dataset_2t.t2m[month_val+i*3,:], month_t) 
    latitudes = dataset_2t.t2m.latitude.values
    years = np.arange(1979, 2019)
    df_temperature = pd.DataFrame(temperatures, index = latitudes, columns = years)
    cmap = plt.cm.get_cmap('seismic', 30)
    plt.figure()
    ax = sns.heatmap(df_temperature, annot = False, cmap=cmap, linewidths=.01, linecolor='lightgray')
    ax.set(xlabel="Years(1979-2018)", ylabel="Latitude", title="Zonal Surface Temperature Deviation %s"%(month))
    # Manually specify colorbar labelling after it's been generated
    colorbar = ax.collections[0].colorbar
    #colorbar.set_ticks([0.25, 1, 1.75])
    #colorbar.set_ticklabels(['Cluster 1', 'Cluster 2', 'Cluster 3'])
    plt.show()
    return df_temperature

def soil_moisture1_mosaic(month):
    """
    0-June, 1-July, 2-August
    """
    if month == 'July':
        month_val = 1
    elif month == 'June':
        month_val = 0
    elif month == 'August':
        month_val = 2
    else:
        print("Do not have data for this particular month\n")
        return
    soil_moistures = np.empty((87,40))
    for i in np.arange(40):
        soil_moistures[:,i] = dataset_wl1.swvl1[month_val+i*3,:]
    latitudes = dataset_2t.t2m.latitude.values
    years = np.arange(1979, 2019)
    df_moisture = pd.DataFrame(soil_moistures, index = latitudes, columns = years)
    cmap = plt.cm.get_cmap('GnBu_r', 15)
    plt.figure()
    ax = sns.heatmap(df_moisture, annot = False, cmap=cmap, linewidths=.01, linecolor='lightgray')
    ax.set(xlabel="Years(1979-2018)", ylabel="Latitude", title="Zonal Soil Water Volumetric Level 1 - %s"%(month))
    # Manually specify colorbar labelling after it's been generated
    colorbar = ax.collections[0].colorbar
    #colorbar.set_ticks([0.25, 1, 1.75])
    #colorbar.set_ticklabels(['Cluster 1', 'Cluster 2', 'Cluster 3'])
    plt.show()
    return df_moisture

def soil_moisture2_mosaic(month):
    """
    0-June, 1-July, 2-August
    """
    if month == 'July':
        month_val = 1
    elif month == 'June':
        month_val = 0
    elif month == 'August':
        month_val = 2
    else:
        print("Do not have data for this particular month\n")
        return
    soil_moistures = np.empty((87,40))
    for i in np.arange(40):
        soil_moistures[:,i] = dataset_wl2.swvl2[month_val+i*3,:]
    latitudes = dataset_2t.t2m.latitude.values
    years = np.arange(1979, 2019)
    df_moisture = pd.DataFrame(soil_moistures, index = latitudes, columns = years)
    cmap = plt.cm.get_cmap('GnBu_r', 15)
    plt.figure()
    ax = sns.heatmap(df_moisture, annot = False, cmap=cmap, linewidths=.01, linecolor='lightgray')
    ax.set(xlabel="Years(1979-2018)", ylabel="Latitude", title="Zonal Soil Water Volumetric Level 2 - %s"%(month))
    # Manually specify colorbar labelling after it's been generated
    colorbar = ax.collections[0].colorbar
    #colorbar.set_ticks([0.25, 1, 1.75])
    #colorbar.set_ticklabels(['Cluster 1', 'Cluster 2', 'Cluster 3'])
    plt.show()
    return df_moisture

In [17]:
#df = temperature_mosaic("August")
#df1 = soil_moisture1_mosaic("August")
#df2 = soil_moisture2_mosaic("June")
df3 = temperature_deviations_mosaic("August")

ValueError: operands could not be broadcast together with shapes (87,) (87, 480)

In [285]:
test = np.arange(0,1,0.1)

In [286]:
test[-1]

0.9

In [287]:
test

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

In [331]:
hum = test[::1]

In [332]:
hum

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

In [290]:
np.append(hum,test[-1])

array([0. , 0.2, 0.4, 0.6, 0.8, 0.9])

In [291]:

hum

array([0. , 0.2, 0.4, 0.6, 0.8])